## Build A Motion Detected Alarm System with Python and OpenCV

### Introduction

In this exercise we are going to develop – again! – a motion triggered alarm based on the frame differencing method. Now, we are going to do this using Python and OpenCV.

Open Source Computer Vision Library (OpenCV) is a library written in C/C++ mainly aimed at real-time computer vision.

OpenCV supports a wide variety of programming languages such as C++, Python, Java, etc. 

In particular, in this exercise we are going to introduce OpenCV-Python, a Python wrapper for the original OpenCV C++ implementation.


The approach for developing this application is the same as proposed in Exercise 14:

When the program starts, the program will capture from the webcam the ‘baseline’ image (see the left image below). This is the image without any 'intruder'. The program will keep capturing frames and comparing them with the baseline image. If no one enters the frame, nothing will happen. However, when somebody or something enters the frame (see the right image below), the program will detect that the captured frame and the baseline image are ‘very’ different and will trigger the audio alarm.

![Alarm](../Images/Alarm_v2.png)

### Exercises

**Very important: you need to run the code in your local machine for displaying the video and using the webcam.**

**Exercise 1.1**: Download (or copy) this Jupyter notebook into your local machine.

**Exercise 1.2**: Install the required Python libraries

                pip install pyttsx3
                pip install pywin32
                pip install numpy
                pip install opencv-python

**Exercise 1.3**: Run and study the code *Capture and display the frames of a video with Python and OpenCV*.

**Exercise 1.4**: Run and study the code *Capture and display webcam with Python and OpenCV*.

**Exercise 1.5**: Run and study the code *Build a motion detected alarm system with Python*.

### Exercise 1.3. Capture and display the frames of a video with Python and OpenCV.

 Run and study the code Capture and display the frames of a video with Python and OpenCV.

In [ ]:
# Capture and display the frames of a video with Python and OpenCV

import cv2

# Create a VideoCapture object and read the frames from an input file
# You can download this video from https://www.loc.gov/item/00694162/ (or use another video!)
video=cv2.VideoCapture('media-ammem-AmericaAtWorkAmericaAtLeisure-1968.mp4')

# Check if the video opened successfully
if (video.isOpened()== False): 
  print("Error opening video file")

# # Read until video is completed or we press 'q'
while True:
    # Capture frame-by-frame
    # Note that VideoCapture captures the frames of a video without considering the fps of the video
    check, frame = video.read()

    if check == True:
    
        cv2.imshow("movie",frame)
    
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break

# After the loop release the video object
video.release()

# Destroy all the windows
cv2.destroyAllWindows()

### Exercise 1.4. Capture and display webcam with Python and OpenCV.

Run and study the code Capture and display webcam with Python and OpenCV.

In [ ]:
# Capture and display webcam with Python and OpenCV

import cv2

# We use VideoCapture function to create the video capture object
# Note we put '0' to capture webcam
video=cv2.VideoCapture(0)

# We start an infinite loop and keep reading frames from the webcam until we press 'q'
while True:
    check, frame = video.read()
    cv2.imshow("Webcam",frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# After the loop release the video object
video.release()

# Destroy all the windows
cv2.destroyAllWindows()

### Exercise 1.5. Build a motion detected alarm system with Python.

Run and study the code Build a motion detected alarm system with Python.

In particular, analyse and try different parameters in the functions:

- GaussianBlur(): for noise reduction (smoothening).
- threshold(): you need to change the threshold function values for a better performance with your webcam , room's light, etc.
- findContours(): this method identifies all the contours in an image. Experiment with the different contour retrieval modes (https://docs.opencv.org/4.5.2/d3/dc0/group__imgproc__shape.html#ga819779b9857cc2f8601e6526a3a5bc71) and contour approximation methods (https://docs.opencv.org/4.5.2/d3/dc0/group__imgproc__shape.html#ga4303f45752694956374734a03c54d5ff),

In [ ]:
# Build A Motion Detected Alarm System with Python

# let’s import the libraries
# For playing the audio, we will be using “pyttsx3” python library to convert text to speech
import cv2
import pyttsx3
import threading
import time


# This funtion plays the audio message
def voice_alarm(alarm_sound):
    alarm_sound.say("Intruder Detected")
    alarm_sound.runAndWait()
    

# Setting parameters for voice
alarm_sound = pyttsx3.init()
voices = alarm_sound.getProperty('voices')
alarm_sound.setProperty('voice', voices[0].id)
alarm_sound.setProperty('rate', 150)


# The function to play the audio wil be executed in a separate thread.
# So, there won't be lag in the video feed while the audio alert message is playing.
alarm = threading.Thread(target=voice_alarm, args=(alarm_sound,))


#status_list=[None,None]
status_list=[0,0]
initial_frame = None


# We use VideoCapture function to create the video capture object
video=cv2.VideoCapture(0)


# We start an infinite loop and keep reading frames from the webcam until we press 'q'
while True:
    check, frame = video.read()
    status=0

    # Gray conversion and noise reduction (smoothening)
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur_frame=cv2.GaussianBlur(gray_frame,(25,25),0)

    
    # The first captured frame is the baseline image
    if initial_frame is None:
        initial_frame = blur_frame
        continue

    # The difference between the baseline and the new frame
    delta_frame=cv2.absdiff(initial_frame,blur_frame)
    # The difference (the delta_frame) is converted into a binary image
    # If a particular pixel value is greater than a certain threshold (specified by us here as 150),
    # it will be assigned the value for White (255) else Black(0)
    # Important: you may have to change the threshold value for a better performance with your webcam , room's light, etc.
    threshold_frame=cv2.threshold(delta_frame,150,255, cv2.THRESH_BINARY)[1]
    

    # The cv2.findContours() method we will identify all the contours in our image.
    # This method expects 3 parameters, (a) image, (b) contour retrieval mode and
    # (c) contour approximation method
    (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        # contourArea() method filters out any small contours
        # You can change this value
        if cv2.contourArea(c) < 1000:
            continue
        status=status + 1
        (x, y, w, h)=cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
    status_list.append(status)


    # The alarm is triggered if an 'intruder' is detected
    # We can also trigger the alarm only if a moving object is detected with
    #if status_list[-1]>= 1 and status_list[-2]==0:    
    if status_list[-2]>= 1:
        if (alarm.is_alive() == False):
            alarm = threading.Thread(target=voice_alarm, args=(alarm_sound,))
            alarm.start()

        
    # To better understand the application, we can visualise the different frames generated
    cv2.imshow('Webcam', frame)
    cv2.imshow('Baseline image', initial_frame)
    cv2.imshow("Gray Frame",gray_frame)
    cv2.imshow('Delta frame', delta_frame)   
    cv2.imshow('Threshold frame', threshold_frame)
    

    # Stop the program by pressing 'q'    
    if cv2.waitKey(1) == ord('q'):
        break

        
# After the loop release the video object, stop the alarm
# and destroy all the windows
alarm_sound.stop()
video.release()
cv2.destroyAllWindows()

Note: This lab is based on https://github.com/arindomjit/Motion_Detected_Alarm by Arindomjit Bhattacharjee.